# 10. Eigenvalue Problem

We talked about that this problem is unstable in general. But it is stable if $A$ is symmetric.

Chapter 9 of my notes

## Applications

- Behavior of ODEs
- Markov Chains (health care economics, Page Rank)
  
  [Rachel's notebook 9 on page rank](https://github.com/fastai/numerical-linear-algebra-v2/blob/master/nbs/09-PageRank-with-Eigen-Decompositions.ipynb)
  
- Spectral Clustering
- quantum mechanics
- SVD!! 

  Name applications?
  
  All that you have mentioned + conditioning of a linear system + the stablest LS solver
- ...

## Page Rank algorithm

### The problem

<img src="./fig/graph1.jpg" style="width: 300px;">

This is a directed graph. The 6 nodes represents 6 pages. An outgoing arrow from 1 to 3 means that there is a link to page 3 on page 1.

In [144]:
import numpy as np
A = np.array([[0,1,0,0,1,1],[0, 0,1,0,1,0],[1,0,0,1,0,1],[0,1,0,0,0,1],[0,1,1,0,0,0],[0,0,1,1,1,0]])
print(A)

[[0 1 0 0 1 1]
 [0 0 1 0 1 0]
 [1 0 0 1 0 1]
 [0 1 0 0 0 1]
 [0 1 1 0 0 0]
 [0 0 1 1 1 0]]


### Simulate clicking

We assume that we start on page 2, and we have equal chance to click on 1, 4, or 5, ...

In [146]:
def Sim_visit(N):
    # N = how many times to click
    PageVisit = np.array([0,1,0,0,0,0]) #store the number of visits of each page
     
    page = 1
    for i in range(N):
        Idx = np.where(A[:,page]==1)[0]
        page = Idx[np.random.randint(len(Idx))]  #randomly go to this page
        PageVisit[page] += 1
    return PageVisit

print(Sim_visit(10000))

[1568 1338 2743 1063 1400 1889]


In [147]:
import pandas as pd
columns = [0,10,100, 1000,10000,20000]
index = range(1,7,1)  #page numbering
Clickdf = pd.DataFrame(index=index, columns=columns)
for i in range(len(columns)):
    clicks = Sim_visit(columns[i])
    Clickdf[columns[i]] = clicks
Clickdf

,0,10,100,1000,10000,20000
1,0,2,15,151,1523,3101
2,1,3,18,139,1389,2730
3,0,2,28,262,2699,5472
4,0,0,11,116,1121,2146
5,0,3,11,141,1349,2684
6,0,1,18,192,1920,3868


In [148]:
# transform to ratio
Ratiodf = Clickdf/[1,11,101,1001,10001,20001]
Ratiodf

,0,10,100,1000,10000,20000
1,0.0,0.181818,0.148515,0.150849,0.152285,0.155042
2,1.0,0.272727,0.178218,0.138861,0.138886,0.136493
3,0.0,0.181818,0.277228,0.261738,0.269873,0.273586
4,0.0,0.000000,0.108911,0.115884,0.112089,0.107295
5,0.0,0.272727,0.108911,0.140859,0.134887,0.134193
6,0.0,0.090909,0.178218,0.191808,0.191981,0.193390


### Using eigenvalue

It turns out that we do not need the Monte Carlo method to find out this ratio. The ratio converges to the eigenvector of?

In [54]:
def ColN(A): #normalize each column such that each column sums up to 1
    n = A.shape[0]
    Colsum = np.sum(A,axis=0)
    return A/np.outer(np.ones(n),Colsum)

In [149]:
np.set_printoptions(suppress=True)
%precision 2
B = ColN(A)
print(B)

[[ 0.    0.33  0.    0.    0.33  0.33]
 [ 0.    0.    0.33  0.    0.33  0.  ]
 [ 1.    0.    0.    0.5   0.    0.33]
 [ 0.    0.33  0.    0.    0.    0.33]
 [ 0.    0.33  0.33  0.    0.    0.  ]
 [ 0.    0.    0.33  0.5   0.33  0.  ]]


$$x^{(1)}=Bx^{(0)}=\begin{bmatrix}
0&1/3&0  &0&1/3&1/3\\
0&0  &1/3&0&1/3&0\\
1&0  &0  &1/2&0&1/3\\
0&1/3&0  &0&0&1/3\\
0&1/3&1/3&0&0&0\\
0&0  &1/3&1/2&1/3&0
\end{bmatrix}\begin{bmatrix}
0\\ 1\\0\\0\\0\\0
\end{bmatrix}=\begin{bmatrix}
1/3\\ 0\\0\\1/3\\1/3\\0
\end{bmatrix},\qquad x^{(2)}=B^2x^{(0)}=Bx^{(1)}=\begin{bmatrix}
0&1/3&0  &0&1/3&1/3\\
0&0  &1/3&0&1/3&0\\
1&0  &0  &1/2&0&1/3\\
0&1/3&0  &0&0&1/3\\
0&1/3&1/3&0&0&0\\
0&0  &1/3&1/2&1/3&0
\end{bmatrix}\begin{bmatrix}
1/3\\ 0\\0\\1/3\\1/3\\0
\end{bmatrix}=\begin{bmatrix}
1/9\\ 1/9\\1/2\\0\\0\\5/18
\end{bmatrix}$$

$B$ is a **probability transition matrix** of this Markov process. $x^{(0)}$ is the initial state vector (nonnegative entries, sums up to 1)

**You have seen such a matrix already in Rachel's notebook 1 ([the health state example](https://nbviewer.jupyter.org/github/fastai/numerical-linear-algebra-v2/blob/master/nbs/01-Why-are-we-here.ipynb#Matrix-Vector-Products:)). Note each row sums up to 1 there.**

$1/9 = 1/3*0 + 1/3*0 + 1/3*1/3$

$\text{Prob(go to page 1) } = \sum_{j=1}^6\text{Prob(currently on page j) }\cdot b_{1j}$

In [150]:
def Anx(A,x0,n): #return A^n(x0)
    x = x0
    for i in range(n):
        x = A@x
    return x

In [151]:
x10 = Anx(B,np.array([0.5,0.5,0,0,0,0]),100)
%precision 6
print(x10)

[ 0.154545  0.136364  0.272727  0.109091  0.136364  0.190909]


#### $\lim_{n\rightarrow\infty}B^nx_0 = $ ranking vector = the eigenvector corresponding to the biggest eigenvalue of $B$

This is the Power method.

In [154]:
# check
e,v = np.linalg.eig(B)
%precision 3
print (v[:,0])
print (v[:,0]/sum(v[:,0]))
print(e)

[-0.360+0.j -0.318+0.j -0.636+0.j -0.254+0.j -0.318+0.j -0.445+0.j]
[ 0.155-0.j  0.136-0.j  0.273-0.j  0.109-0.j  0.136-0.j  0.191-0.j]
[ 1.000+0.j    -0.298+0.552j -0.298-0.552j  0.184+0.j    -0.255+0.j
 -0.333+0.j   ]


### A more complicated model

<img src="./fig/graph2.jpg" style="width: 400px;">

The cluster of Pages 1, 2, 3 has no outgoing links to the cluster of Pages 4, 5, 6, so once a surfer exits cluster 4,5,6, the surfer will be "trapped" in cluster 1,2,3 and the faractional page counts for pages 4,5,6 will approach zero, thereby assignning the pages in that cluster a page rank of 0.

In [64]:
A2 = np.array([[0,1,0,0,1,1],[0, 0,1,0,1,0],[1,1,0,1,0,1],[0,0,0,0,0,1],[0,0,0,0,0,0],[0,0,0,1,1,0]])
print (A2)

[[0 1 0 0 1 1]
 [0 0 1 0 1 0]
 [1 1 0 1 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]
 [0 0 0 1 1 0]]


In [65]:
B2 = ColN(A2)
print(B2)

[[ 0.        0.5       0.        0.        0.333333  0.333333]
 [ 0.        0.        1.        0.        0.333333  0.      ]
 [ 1.        0.5       0.        0.5       0.        0.333333]
 [ 0.        0.        0.        0.        0.        0.333333]
 [ 0.        0.        0.        0.        0.        0.      ]
 [ 0.        0.        0.        0.5       0.333333  0.      ]]


In [67]:
print (Anx(B2,np.ones(6)/6,100))

[ 0.2  0.4  0.4  0.   0.   0. ]


Solution: with probability $\delta$ that the surfer chooses a link, and with probability $1-\delta$ that the surfer choose the next page at random. In the latter case, the surfer will choose any particular page at random with prob $\frac{1-\delta}{n}$.

Prob (pick i while on j) = $m_{ij} = \delta b_{ij}+\frac{1-\delta}{n}$

$M = \delta B+\frac{1-\delta}{n}\begin{bmatrix}
1&1&\cdots&1\\
1&1&\cdots&1\\
\vdots&\vdots&\ddots&\vdots\\
1&1&\cdots&1\\
\end{bmatrix}$

In [69]:
delta = 0.85 #google's choice
n = A2.shape[0]
M = delta*B2 + (1-delta)/n*np.outer(np.ones(n),np.ones(n))
print (Anx(M,np.ones(6)/6, 100)) #M^n(x0)

[ 0.189539  0.338113  0.360035  0.038757  0.025     0.048555]


### This is a place where you want to use sparse matrix mode

In [88]:
N = 10000  #number of webpages
k = 20 #each page has 1-k links
X = np.zeros([N,N])
for i in range(N):
    n = np.random.randint(k) + 1  #this number is between 1 and k
    Index = np.random.randint(N,size = n)
    Val = np.random.rand(n)
    X[Index,i] = Val
X = ColN(X)

In [89]:
import psutil
import os
def mem_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / psutil.virtual_memory().total

In [90]:
mem_usage()

0.078034

In [80]:
#convert it to CSR
from scipy.sparse import csr_matrix
X = csr_matrix(X)

In [81]:
X

<10000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 104586 stored elements in Compressed Sparse Row format>

In [87]:
mem_usage()

0.005136

Compare speed

In [91]:
N = 10000  #number of webpages
k = 20 #each page has 1-k links
X = np.zeros([N,N])
for i in range(N):
    n = np.random.randint(k) + 1  #this number is between 1 and k
    Index = np.random.randint(N,size = n)
    Val = np.random.rand(n)
    X[Index,i] = Val
X = ColN(X)

In [92]:
SX = csr_matrix(X)

In [101]:
import timeit
x0 = np.ones(N)/N
%time Anx(X,x0,10)   #X^10(x0)

CPU times: user 1.28 s, sys: 13 ms, total: 1.29 s
Wall time: 683 ms


array([ 0.000084,  0.000066,  0.000099, ...,  0.000063,  0.000083,
        0.000125])

In [103]:
import timeit
x0 = np.ones(N)/N
%time Anx(SX,x0,10) 

CPU times: user 4.02 ms, sys: 943 µs, total: 4.96 ms
Wall time: 3.73 ms


array([ 0.000084,  0.000066,  0.000099, ...,  0.000063,  0.000083,
        0.000125])

## Power Method

for computing the largest eigenpair

### The idea is simple

we assume $A$ is diagonalizable and $|\lambda_1|>|\lambda_2|\geq\cdots\geq|\lambda_n|$. Let $Av_i = \lambda_iv_i , i = 1, ..., n$

Diagonalizable = able to find $n$ independent eigenvetors.

Given arbitrary $x$, it can be expressed as a linear combination of $A$'s eigenbasis: $x=\sum_{i=1}^nc_iv_i$, then

$$A^kx=\sum_{i=1}^nc_iA^kv_i=\sum_{i=1}^nc_i\lambda_i^kv_i\Longrightarrow \frac{1}{\lambda_1^k}A^kx=c_1v_1+\sum_{i=2}^nc_i\left(\frac{\lambda_i}{\lambda_1}\right)^kv_i$$

Convergence rate is dominated by the 2nd biggest eigenvalue.

**Example 1: **

Suppose $A$ is 5 by 5 and has eigenvalues 10, -9, 8, 7, 6.  Then the power method will find the eigenvalue 10 with a linear rate $9/10=0.9$.

|The Power Method||
|:---|
|Initialize $v^{(0)}$ such that $\|v^{(0)}\|_2=1$||
|for some termination criteria:||
|$v^{(k)}=\frac{Av^{(k-1)}}{\|Av^{(k-1)}\|}$|apply $A$|
|$\lambda^{(k)}=(v^{(k)})^TAv^{(k)}$|Rayleigh quotient|


```diff
-Although implemented in page rank algorithm,  it is by no means an effective tool for general use.
```

### However, it is useful for finding eigenvectors if eigenvalue is known



#### Example 1 continued:



Suppose $A$ is 5 by 5 and has eigenvalues 10, -9, 8, 7, 6.

Then the eigenvalues of $A-7.9I$ are? 2.1, -16,9, 0.1, -0.9, -1.9

Then the eigenvalues of $(A-7.9I)^{-1}$ are? 

$\frac{1}{2.1}, \frac{-1}{16.9}, \frac{1}{0.1}, \frac{-1}{0.9}, \frac{-1}{1.9}$

Power method on $(A-7.9I)^{-1}$ will converge to what eigenvalue at what rate?

converge to $\frac{1}{0.1}$ at the rate $0.1/0.9=\frac{1}{9}$


#### Inverse Iteration for finding the eigenvector corresponding to approximately known eigenvalue 

pick $\mu$ to be close to the known eigenvalue, and keep applying $(A-\mu I)^{-1}$ to some initial vector.

How?

|Inverse Iteration (used in practice)||
|:---|:---|
|Initialize $v^{(0)}$ such that $\|v^{(0)}\|_2=1$||
|for some termination criteria:||
|Solve $(A-\mu I)w=v^{(k-1)}$ for $w$|apply $(A-\mu I)^{-1}$|
|$v^{(k)} = w/\|w\|$|normalize|

## Review on eigenvalue related material

### For what matrices are the eigenvalues easy to compute?

### Eigenvalue revealing decompositions

#### $A=UBU^{-1}$

What is a diagonalizable matrix?

Answer: there exist a diagonal matrix $D$, and an invertible matrix $U$ such that $A= UDU^{-1}$

$AU=UD$

#### $A = QDQ^{-1}$ where $Q$ is orthogonal and $D$ is diagonal

What is an orthogonal matrix?

$A$ must be what kind of matrix?

Intuition on why finding this kind of matrices' eigenvalues is more stable.

#### Schur decomposition: $A = QTQ^{-1}$ where $Q$ is orthogonal and $T$ is upper triangular

If $A$ is symmetric, then $T$ is forced to be?

**This decomposition can be done to every square matrix. We can read the eigenvalues from $T$.**



## Two Phases of Eigenvalue Computations

Finall we get to talk about how to find eigenvalues in practice

### Phase I: Obtain a Hessenberg form through similarity transformation

Householder reflector is used

$\left[\begin{array}{ccccc}
*&*&*&*&*\\
\times&*&*&*&*\\
\times&*&*&*&*\\
\times&*&*&*&*\\
\times&*&*&*&*
\end{array}\right]$  $\rightarrow$  $\left[\begin{array}{ccccc}
*&*&*&*&*\\
*&*&*&*&*\\
&\times&*&*&*\\
&\times&*&*&*\\
&\times&*&*&*
\end{array}\right]$  $\rightarrow$  $\left[\begin{array}{ccccc}
*&*&*&*&*\\
*&*&*&*&*\\
&*&*&*&*\\
&&\times&*&*\\
&&\times&*&*
\end{array}\right]$  $\rightarrow$  $\left[\begin{array}{ccccc}
*&*&*&*&*\\
*&*&*&*&*\\
&*&*&*&*\\
&&*&*&*\\
&&&*&*
\end{array}\right]$

$\hspace{1.5cm}$ $A_0=A $ $\hspace{1.8cm}$ $A_1=Q_1A_0Q_1^T$ $\hspace{1.8cm}$ $A_2=Q_2A_1Q_2^T$ $\hspace{1.5cm}$ $B=Q_3A_2Q_3^T$ 

The process of finding $Q_i$ is very similar to finding the $Q_i$ in QR decomposition. Details see Section 9.2.

What will happen if $A$ is symmetric?

The Hessenberg form (not upper triangular) is the best we can get. Why?



In [107]:
from scipy.linalg import hessenberg
?hessenberg

### Phase II: QR algorithm is the mainstream algorithm

#### The Pure QR

    Initialize A_0 = A
    for k = 1,2,...
    Q_k*R_k = A_k-1
    A_k = R_kQ_k
        

Anlysis: $A^{(k)}=R^{(k)}Q^{(k)}=[Q^{(k)}]^TQ^{(k)}R^{(k)}Q^{(k)}=[Q^{(k)}]^TA^{(k-1)}Q^{(k)}$

In [110]:
import numpy as np
# inputs: M is a square matrix; K is number of iterations.
def QR_pure(M,K): 
    for i in range(K):
        Q,R = np.linalg.qr(M)
        M = R@Q
    return M

#### The shifted QR - used in practice

     Initialize A_0 = A
     for k = 1,2,...
     pick a shift mu_k  #standard choice is A_k-1[m,m]
     Q_k*R_k = A_k-1 - mu_k*I
     A_k = R_kQ_k + mu_k*I

In [142]:
from scipy.linalg import qr,eig,eigvals
def QR_shift(M,K): 
    n = M.shape[0]
    for i in range(K):
        mu = M[-1,-1]  #this is a choice given in Algorithm 28.2 of Trefethen and Bau
        
        Q,R = qr(M - mu*np.eye(n))
        M = R@Q + mu*np.eye(n)
    return M  

### Put two phases together

In [113]:
A = np.array([[5.0,-4,-6,9],[-1,2,6,-3],[2,3,-5,4],[4,1,-1,0]]); print(A)

[[ 5. -4. -6.  9.]
 [-1.  2.  6. -3.]
 [ 2.  3. -5.  4.]
 [ 4.  1. -1.  0.]]


In [114]:
H = hessenberg(A); print(H)

[[ 5.        6.110101  6.87139  -6.960652]
 [ 4.582576 -0.190476  5.197377 -0.898964]
 [ 0.       -1.005653 -6.908179  2.697878]
 [ 0.        0.       -0.575391  4.098655]]


In [116]:
print(QR_shift(H,100))

[[-6.        2.112068 -7.706925  4.362495]
 [-0.       -4.        4.216043 -5.354671]
 [ 0.        0.        8.       -4.320138]
 [ 0.        0.        0.        4.      ]]


In [124]:
W= eigvals(A); print(W)

[-6.+0.j -4.+0.j  8.+0.j  4.+0.j]


### Complexity of this two phase algorithm

$\left[\begin{array}{ccccc}
*&*&*&*&*\\
*&*&*&*&*\\
*&*&*&*&*\\
*&*&*&*&*\\
*&*&*&*&*
\end{array}\right]\xrightarrow[direct\ methods]{Phase\ I}\left[\begin{array}{ccccc}
*&*&*&*&*\\
*&*&*&*&*\\
&*&*&*&*\\
&&*&*&*\\
&&&*&*
\end{array}\right]\xrightarrow[iterative\ methods]{Phase\ II}\left[\begin{array}{ccccc}
*&*&*&*&*\\
&*&*&*&*\\
&&*&*&*\\
&&&*&*\\
&&&&*
\end{array}\right]$

||General matrices|Symmetric matrices|
|:---|:---|:---|
|Phase I|$O(n^3)$|$O(n^3)$ (half)|
|Phase II|$O(n^3)$|$O(n^2)$|
|Total|$O(n^3)$|$O(n^3)$|
|(Phase II without Phase I)|$O(n^4)$ or higher||


Symmetric matrices come up naturally:

- Least Squres problem and finding singular values
- Distance matrices
- Relationship matrices (Facebook or LinkedIn)
- PDEs

## Finding SVD $A = U\Sigma V^T$

rather complicated...

$A^TA =V\Sigma^T\Sigma V^T$

Mathematically, we can find SVD of $A$ as follows:

1. Form $A^TA$
2. Compute the eigenvalue decomposition $A^TA = V\Lambda V^T$
3. Let $\Sigma$ be the $m\times n$ nonnegative diagonal square root of $\Lambda$.
4. Solve the system $U\Sigma=AV$ for orthogonal $U$.

```diff
-This is however unstable.
```

### The stable way:

Assume $A$ is square for the moment. Construct

$H=\begin{bmatrix}0 & A^T\\A & 0\end{bmatrix}$

If $A=U\Sigma V^T$, then $AV=U\Sigma$ and $A^TU=V\Sigma$, so we can easily check that

$\begin{bmatrix}0 & A^T\\A & 0\end{bmatrix}\begin{bmatrix}V & V\\U & -U\end{bmatrix}=\begin{bmatrix}V & V\\U & -U\end{bmatrix}\begin{bmatrix}\Sigma & 0\\0 & -\Sigma\end{bmatrix}$

See Lecture 31 of Trefethan and Bau for details.